# Preprocessing

In [1]:
import json
import uuid

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from difflib import SequenceMatcher
from IPython.display import HTML

from utils.code_utils import tokenize_python_code

pd.set_option('display.max_columns', None)

codenet_root = 'datasets/codenetpy/'

In [2]:
with open(codenet_root + 'codenetpy_train.json', 'r') as f:
    data = json.load(f)["data"]
    
with open(codenet_root + 'codenetpy_test.json', 'r') as f:
    test_data = json.load(f)["data"]
    

train_df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)

codenetpy_df = pd.DataFrame(data + test_data)

display(codenetpy_df)
display(codenetpy_df.info())

,original_src,changed_src,problem_id,original_id,changed_id,language,filename_ext,original_status,returncode,error_class,error_class_extra,error,output
0,l=[]\nwhile True:\n try:\n l.append(...,l=[]\nwhile True:\n try:\n l.append(...,p00001,s196059089,s508355022,Python,py,Runtime Error,1,SyntaxError,SyntaxError: Missing parentheses in call to 'p...,"File ""/home/alex/Documents/research/bug-dete...",
1,"data = []\nfor i in range(0,10):\n data.app...","data = []\nfor i in range(0,10):\n data.app...",p00001,s840313913,s627573591,Python,py,WA: Presentation Error,0,0,,,\n3776\n2848\n2840\n
2,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,p00001,s973623080,s521689663,Python,py,Runtime Error,1,TypeError,TypeError: '>' not supported between instances...,"Traceback (most recent call last):\n File ""/h...",
3,for i in range(10):\n s.append(int(input())...,s=[]\nfor i in range(10):\n s.append(int(in...,p00001,s393051088,s459982723,Python,py,Runtime Error,1,NameError,NameError: name 's' is not defined,"Traceback (most recent call last):\n File ""/h...",
4,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,p00001,s803831828,s403126268,Python,py,Runtime Error,1,NameError,NameError: name 'argv' is not defined,"Traceback (most recent call last):\n File ""/h...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54587,"N = int(input())\nL = list(map(int, input().sp...","N = int(input())\nL = list(map(int, input().sp...",p04047,s420396881,s036877571,Python,py,Runtime Error,1,IndexError,IndexError: list index out of range,"Traceback (most recent call last):\n File ""/h...",
54588,\nnum = int(input())\nli = int(input().split()...,n = int(input())\nli = [int(x) for x in input(...,p04047,s838662080,s028090472,Python,py,Runtime Error,1,TypeError,"TypeError: int() argument must be a string, a ...","Traceback (most recent call last):\n File ""/h...",
54589,from collections import Counter\n \nn = int(in...,"n = int(input())\n \narr = list(map(int, input...",p04047,s112597968,s888016571,Python,py,Runtime Error,1,TypeError,TypeError: unsupported operand type(s) for +=:...,"Traceback (most recent call last):\n File ""/h...",
54590,"N,X = input().split()\nN,X = int(N), int(X)\na...","N,X = input().split()\nN,X = int(N), int(X)\na...",p04048,s311100241,s323769427,Python,py,Runtime Error,1,TabError,TabError: inconsistent use of tabs and spaces ...,"File ""/home/alex/Documents/research/bug-dete...",


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54592 entries, 0 to 54591
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_src       54592 non-null  object
 1   changed_src        54592 non-null  object
 2   problem_id         54592 non-null  object
 3   original_id        54592 non-null  object
 4   changed_id         54592 non-null  object
 5   language           54592 non-null  object
 6   filename_ext       54592 non-null  object
 7   original_status    54592 non-null  object
 8   returncode         54592 non-null  int64 
 9   error_class        54592 non-null  object
 10  error_class_extra  54592 non-null  object
 11  error              54592 non-null  object
 12  output             54592 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.4+ MB


None

In [3]:
codenetpy_df['error_class'].value_counts()

0                            28406
SyntaxError                   8686
NameError                     5262
TypeError                     4202
ValueError                    2444
IndentationError              1081
IndexError                     999
AttributeError                 913
EOFError                       852
TLEError                       525
ModuleNotFoundError            357
TabError                       265
ImportError                    132
ZeroDivisionError               79
KeyError                        66
SyntaxWarning                   62
FileNotFoundError               62
UnboundLocalError               61
1                               58
DeprecationWarning              35
RecursionError                  13
OverflowError                   12
RuntimeError                     6
-11                              5
2                                3
OSError                          3
SparseEfficiencyWarning          1
255                              1
VisibleDeprecationWa

In [4]:
codenetpy_df['returncode'].value_counts()

 0      28455
 1      25603
-9        525
-11         5
 2          3
 255        1
Name: returncode, dtype: int64

In [5]:
# There are cases where the return code is 0 but the error_class is a Warning

codenetpy_df[(codenetpy_df['returncode'] == 0) & (codenetpy_df['error_class'] != '0')]['error_class'].value_counts()

DeprecationWarning         30
SyntaxWarning              18
SparseEfficiencyWarning     1
Name: error_class, dtype: int64

In [6]:
# Most of the '0' error_class buggy submissions are TLE errors, which means they are inefficient implementations

codenetpy_df[codenetpy_df['error_class'] == '0']['original_status'].value_counts()

Time Limit Exceeded       17252
Runtime Error              9590
WA: Presentation Error     1485
Memory Limit Exceeded        73
Output Limit Exceeded         5
Judge Not Available           1
Name: original_status, dtype: int64

In [7]:
# Idealy the model will take as input the original_src code and will identify both the error_description and the location of the bug
# We can achieve this by comparing the original_src with the changed_src, which will give us the location of the bug (e.g. the modification)
# Next we will want to create a new model that will take as input the original_src, and the found bug and will learn to make the modifications
# on the original code such that it gets accepted
buggy_df = codenetpy_df[codenetpy_df['returncode'] != 0]
accepted_df = codenetpy_df[codenetpy_df['returncode'] == 0]

# renombrar columnas
buggy_df.rename(columns={'original_src': 'code_string', 'error_class_extra': 'error_msg'}, inplace=True)
accepted_df.rename(columns={'changed_src': 'code_string'}, inplace=True)

# filtrado de columnas
buggy_df = buggy_df[['code_string', 'error_msg']]
accepted_df = accepted_df[['code_string']]

# generacion de uuids
buggy_df['uuid'] = [uuid.uuid4().hex for _ in range(len(buggy_df.index))]
buggy_df = buggy_df.set_index('uuid')
accepted_df['uuid'] = [uuid.uuid4().hex for _ in range(len(accepted_df.index))]
accepted_df = accepted_df.set_index('uuid')
#buggy_df


/Users/angelchavez/opt/miniconda3/envs/BIFI/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
buggy_df.to_json(r'preprocess/raw.bad.json', orient='index', indent=2)
accepted_df.to_json(r'preprocess/raw.good.json', orient='index', indent=2)

# Tokenizing

In [9]:
raw_bad_df = pd.read_json('preprocess/raw.bad.json', orient='index')
raw_good_df = pd.read_json('preprocess/raw.good.json', orient='index')
#raw_df['code_toks_joined'], raw_df['anonymize_dict'] = zip(*raw_df['code_string'].map(tokenize_python_code))
display(raw_bad_df.isnull().sum())
display(raw_good_df.isnull().sum())


code_string    0
error_msg      0
dtype: int64

code_string    0
dtype: int64

In [10]:
raw_bad_df[['code_toks_joined', 'anonymize_dict']] = raw_bad_df.apply(lambda row: tokenize_python_code(row['code_string']), axis='columns', result_type='expand')
raw_bad_df['code_toks_joined'] = raw_bad_df['code_toks_joined'].str.join(" ")
display(raw_bad_df.isnull().sum())

raw_good_df[['code_toks_joined', 'anonymize_dict']] = raw_good_df.apply(lambda row: tokenize_python_code(row['code_string']), axis='columns', result_type='expand')
raw_good_df['code_toks_joined'] = raw_good_df['code_toks_joined'].str.join(" ")
display(raw_good_df.isnull().sum())


('ERRORTOKEN', (2, 40))
('ERRORTOKEN', (7, 20))
('ERRORTOKEN', (6, 8))
('ERRORTOKEN', (5, 13))
('ERRORTOKEN', (3, 0))
('ERRORTOKEN', (5, 8))
('ERRORTOKEN', (7, 0))
('ERRORTOKEN', (4, 0))
('ERRORTOKEN', (16, 6))
('ERRORTOKEN', (7, 39))
('ERRORTOKEN', (12, 8))
('ERRORTOKEN', (7, 9))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (4, 9))
('ERRORTOKEN', (12, 40))
('ERRORTOKEN', (3, 4))
('ERRORTOKEN', (6, 27))
('ERRORTOKEN', (3, 19))
('ERRORTOKEN', (1, 53))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (16, 27))
('ERRORTOKEN', (4, 31))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (1, 2))
('ERRORTOKEN', (4, 6))
('ERRORTOKEN', (12, 31))
('ERRORTOKEN', (13, 78))
('ERRORTOKEN', (1, 19))
('ERRORTOKEN', (5, 0))
('ERRORTOKEN', (14, 27))
('ERRORTOKEN', (17, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (5, 11))
('ERRORTOKEN', (2, 27))
('ERRORTOKEN', (22, 21))
('ERRORTOKEN', (20, 4))
('EOF in multi-line string', (

code_string           0
error_msg             0
code_toks_joined    577
anonymize_dict        0
dtype: int64

code_string         0
code_toks_joined    0
anonymize_dict      0
dtype: int64

In [11]:
#raw_df.replace(np.nan,'',regex=True)
#raw_df.isnull().sum()
#raw_df_non_null = raw_df.fillna("")
raw_filtered_bad_df = raw_bad_df.dropna(how='any',axis=0) 
raw_filtered_good_df = raw_good_df.dropna(how='any',axis=0) 
display(raw_filtered_bad_df.head())
display(raw_filtered_good_df.head())
#raw_df.head()

,code_string,error_msg,code_toks_joined,anonymize_dict
8b8cbfd1774a4159affe3634398b1a6f,l=[]\nwhile True:\n try:\n l.append(...,SyntaxError: Missing parentheses in call to 'p...,l = [ ] <NEWLINE> while True : <NEWLINE> <INDE...,{}
c083f900127e479d8f42c77e3d1f5ca4,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,TypeError: '>' not supported between instances...,first = 0 ; <NEWLINE> second = 0 ; <NEWLINE> t...,{}
148db506d298471ba5555201856627af,for i in range(10):\n s.append(int(input())...,NameError: name 's' is not defined,for i in range ( 10 ) : <NEWLINE> <INDENT> s ....,{}
de20f382e36f4a30ace5889baea28104,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,NameError: name 'argv' is not defined,<COMMENT> <NL> <NL> import sys <NEWLINE> <NL> ...,"{'<COMMENT>': ['# -*- coding: utf-8 -*-'], '<S..."
2c7b8f8e5db54324a1a3e2a9e15f8fc1,height = []\nwhile 1:\n h = raw_input()\n ...,SyntaxError: Missing parentheses in call to 'p...,height = [ ] <NEWLINE> while 1 : <NEWLINE> <IN...,{'<STRING>': ['''']}


,code_string,code_toks_joined,anonymize_dict
000439bfa6b64a7590adc9a47fde13d0,"N,x,MOD = map(int, input().split())\ns = x\nfl...","N , x , MOD = map ( int , input ( ) . split ( ...",{}
0004705a64d84525aafe253b687139eb,cards = int(input())\ni = 0\ncardlist = []\nwh...,cards = int ( input ( ) ) <NEWLINE> i = 0 <NEW...,"{'<STRING>': ['""S""', '""H""', '""C""', '""D""', '""S ..."
00056ae6734b49fc912adfb1d8d642e6,"def f(B):\n if isinstance(B, float):\n ...",def f ( B ) : <NEWLINE> <INDENT> if isinstance...,{}
0005706be4c94098bb86be8c287feeea,def main():\n N = int(input())\n A = lis...,def main ( ) : <NEWLINE> <INDENT> N = int ( in...,{}
00063399a0084db5a0b6fd27ad5f3607,"N,Q = (int(X) for X in input().split())\nS = i...","N , Q = ( int ( X ) for X in input ( ) . split...",{'<STRING>': [''AC'']}


In [12]:
raw_filtered_bad_df.to_json(r'preprocess/tokenized.bad.json', orient='index', indent=2)
raw_filtered_good_df.to_json(r'preprocess/tokenized.good.json', orient='index', indent=2)


## Tratamiento adicional de JSON

In [13]:
with open('preprocess/tokenized.bad.json', 'r') as f1:
    data_bad = json.load(f1)


In [14]:
for key, obj in data_bad.items():
    for attribute, value in obj.items():
        if attribute == 'error_msg':
            new_obj = { 'msg': value }
            obj[attribute] = new_obj
            
# Serializing json
json_object_bad = json.dumps(data_bad, indent=2)
# Writing to final.bad.json.json
with open("preprocess/orig_bad_code/final.bad.json", "w") as outfile1:
    outfile1.write(json_object_bad)


In [15]:
with open('preprocess/tokenized.good.json', 'r') as f2:
    data_good = json.load(f2)

json_object_good = json.dumps(data_good, indent=2)

with open("preprocess/orig_good_code/final.good.json", "w") as outfile2:
    outfile2.write(json_object_good)